<요약>
- CLASS torch.nn.Module(*args, **kwargs)
- 변수
  - training (bool): 해당 모듈 학습 여부를 Boolean으로 나타낸다. 
  - add_module(name, module): 현재 모듈에 자식 모듈을 삽입한다. 모듈은 name을 통해 속성처럼 접근할 수 있다.
  - apply(fn): fn를 자신(self)를 포함한 모든 서브 모델에 재귀적으로 적용한다. [Code]
  - *bfloat16()*: 파라미터와 버퍼를 bfloat16 타입으로 캐스팅 한다
  - `buffers(recurse=True): 모듈 버퍼에 대한 이터레이터 리턴` [Code]
  - `children(): 자식 모듈의 이터레이터를 반환`
  - compile(*args, **kwargs): 모듈의 forward를 torch.compile()를 이용해서 컴파일 한다. 이 모듈의 __call__ 메소드가 컴파일 되고, 모든 인수가 torch.compile()에 전달된다.
  - cpu(): 파라미터와 버퍼를 cpu로 이동
  - gpu(): 파라미터와 버퍼를 gpu로 이동. 또한, 파라미터와 버퍼를 다른 객체로 만든다. `따라서, 모듈이 최적화 되는 동안 GPU에 있는 경우 Optimzer를 구성하기전에 호출되어야 한다(?)`
  - *double()*: 파라미터와 버퍼를 double() 타입으로 캐스팅 한다.
  - `eval(): 모듈을 평가 모드로 설정, 특정 모듈에만 영향을 미친다.` [Code]
  - extra_repr(): 모듈의 추가 표현 설정, 사용자 정의된 추가 정보를 인쇄하려면 자체 모듈에서 이 메서드를 다시 구현해야 합니다. 한 줄 문자열과 여러 줄 문자열 모두 허용.
  - *float()*: 파라미터와 버퍼를 float() 타입으로 캐스팅 한다.
  - forward(*input): 모든 호출에서 수행되는 계산 정의. 모든 하위 클래스에서 Overriden 되어야 한다.
    - 전방향에 대한 레시피는 이 함수 내에서 정의되어야 하지만, Former는 등록된 Hooks 실행을 처리하고 Latter는 자동으로 무시되므로 나중에 Module 객체를 호출해야 한다(?) 
  - get_buffer(target): target이 제공한 버퍼가 있으면 반환하고, 그렇지 않으면 오류 발생
  - get_extra_state()
  - get_parameter(target): target이 제공한 파라미터가 있으면 반환하고, 그렇지 않으면 오류 발생
  - get_submodule(target): target이 제공한 서브모듈이 있으면 반환하고, 그렇지 않으면 오류 발생
  - register_buffer(name, tensor, persistent=True): 모듈에 버퍼를 추가 [Code]
  - ..
  - modules(): 네트워크의 모든 모듈의 이터레이터 반환 [Code]
  - ..
  - zero_grad(set_to_none=True): 모든 모델 파라미터의 기울기를 재 성정. torch.optim.Optimizer와 유사 기능

<알아야 하는 것>
- modules(), parameters(), buffers()

#### CLASS torch.nn.Module(*args, **kwargs)
- https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_buffer
- 모든 뉴럴 네트워크 모듈의 베이스 클래스이다.
- 구현자의 모델은 모두 본 클래스의 자식 클래스이다.
- 모듈은 다른 모듈을 포함(contain)할 수 있고, 이 들을 트리 구조로 중첩 시킬수 있다. 서브 모듈을 attributes 처럼 사용할 수 있다.

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))


- 서브 모듈(위의 예제에서는 nn.Conv2d())도 등록이 되며 to() 를 이용하여 호출할 때 해당 매개변수도 변환된다.
- 상위 클래스(nn.Module)에 대한 호출 (super().__init__())은 하위 클래스 생성 전에 이뤄저야 한다.

#### 변수  

`apply(fn)`
- fn를 자신(self)를 포함한 모든 서브 모델에 재귀적으로 적용한다.

In [16]:
@torch.no_grad()
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight, '<-----')
net = nn.Sequential(nn.Linear(2,2), nn.Linear(2,2))
print(net)
print('\n')
net.apply(init_weights)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True) <-----
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True) <-----
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

##### `buffers(recurse=True)`
- 모듈 버퍼에 대한 이터레이터 리턴

In [5]:
model = Model()
for buf in model.buffers():
    print(type(buf, buf.size()))

#### `eval()`
- 모듈을 평가 모드로 설정
- Dropout, BatchNorm, etc 과 같은 특정 모듈에만 영향을 미칩니다. 동작에 대한 자세한 내용은 특정 모듈의 문서를 참조.
- train(False)와 등가이다.

#### `get_submodule(target)`
- 

#### `named_parameters(prefix='', recurse=True, remove_duplicate=True)`
- 모듈 파라미터의 이터레이터를 리턴한다. 

#### `modules()`
- 네트워크 상의 모든 모듈의 이터레이터를 반환

In [5]:
import torch.nn as nn
l = nn.Linear(2, 2)
net = nn.Sequential(l, l)
print(net)

print('-' * 50)

for idx, m in enumerate(net.modules()):
    print(idx, '->', m)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)
--------------------------------------------------
0 -> Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)
1 -> Linear(in_features=2, out_features=2, bias=True)


#### `register_buffer(name, tensor, persistent=True)`
- 모듈에 버퍼를 추가
- 파라미터로 간주되어서는 안되는 버퍼를 등록
  - 예를 들어 BatchNorm의 `running_mean`은 매개 변수가 아니지만, 모듈 상태의 일부.
  - 버퍼는 디폴트로 영구적이며, 파라미터와 함께 저장된다.
- 영구 버퍼와 비영구 버퍼(persistent=False)의 유일한 차이점은 (persistent=False) 하는 경우 버퍼는 state_dict에 포함되지 않게 된다.
- 버퍼는 파라미터와 마찬가지로 이름을 사용하여 속성 처럼 접근할 수 있다.

In [ ]:
self.register_buffer('running_mean', torch.zeros(num_features))

#### `modules()`
- 네트워크의 모든 모듈의 이터레이터 반환

In [12]:
l = nn.Linear(2, 2)
net = nn.Sequential(l, l)
for idx, m in enumerate(net.modules()):
    print(idx, '->', m)

0 -> Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)
1 -> Linear(in_features=2, out_features=2, bias=True)


#### `zero_grad(set_to_none=True)`
- 모든 모델 파라미터의 기울기를 재 성정. torch.optim.Optimizer와 유사 기능

Q. 모듈과 파라미터 차이??
- 모듈은 모델을 구성하는 클래스 집합 인 것으로 보이고,
- 파라미터는 학습 가능한 파라미터를 의미한다. 

In [7]:
import torchvision
model = torchvision.models.resnet18(pretrained=False)

In [8]:
for idx, m in model.modules(): # named_modules()로 호출하지 않으니 불러 오지 못한다??
    print(m)
    print('=' * 50)

TypeError: cannot unpack non-iterable ResNet object

In [ ]:

for idx, m in model.named_modules():
    print(m)
    print('=' * 50)
    for param in m.parameters():
        print(type(param), param.size())
    print('=' * 50)

----

#### Register_Buffer 상세
- nn.Module.register_buffer('attribute_name', t)
  - 모듈 내에서 tensor t는 self.attribute_name 으로 접근 가능하다.
  - Tensor t는 학습되지 않는다. (중요)
  - model.cuda() 시에 t도 함께 GPU로 간다.

In [10]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.param = nn.Parameter(torch.randn([2, 2]))
        
        self.register_buffer('buff', torch.randn([2, 2]))

        self.non_buff = torch.rand([2, 2])

    def forward(self, x):
        return x

model = Model()
print(model.param.device)
print(model.buff.device)
print(model.non_buff.device)

print()

print(model.state_dict())



cpu
cpu
cpu

OrderedDict([('param', tensor([[ 0.3470, -1.1370],
        [-1.2748,  1.4021]])), ('buff', tensor([[-0.1974, -0.8364],
        [ 0.8150,  1.5108]]))])


In [11]:
model.cuda()
print(model.param.device)
print(model.buff.device)     # buffer는 GPU로 넘어가지만, 학습에 참여하진 않는다!!
print(model.non_buff.device) # 파라미터만 GPU에 넘어간다!!

print()

print(model.state_dict())

cuda:0
cuda:0
cpu

OrderedDict([('param', tensor([[ 0.3470, -1.1370],
        [-1.2748,  1.4021]], device='cuda:0')), ('buff', tensor([[-0.1974, -0.8364],
        [ 0.8150,  1.5108]], device='cuda:0'))])


In [18]:
for name, param in model.named_parameters():
    print(name) # 'param'만 parameter 이다. 
    print(param)
    print(param.data) 

param
Parameter containing:
tensor([[ 0.3470, -1.1370],
        [-1.2748,  1.4021]], device='cuda:0', requires_grad=True)
tensor([[ 0.3470, -1.1370],
        [-1.2748,  1.4021]], device='cuda:0')


In [17]:
for name, buff in model.named_buffers():
    print(name) # 'buff'만 buffer 이다.
    print(buff) # gpu에 적재 되었지만, 학습 대상이 아니다. optimizer로 업데이트 되지 않는다. requires_grad=True가 없다.

buff
tensor([[-0.1974, -0.8364],
        [ 0.8150,  1.5108]], device='cuda:0')
